In [1]:
import os
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '../python')


import time
import matplotlib.pyplot as plt
from matplotlib.widgets import Button

In [2]:
import time
import matplotlib.pyplot as plt
# from matplotlib.widgets import Button

from utils import map_vis_without_lanelet
from utils import dataset_reader
from utils import dataset_types
from utils import map_vis_lanelet2
from utils import tracks_vis
from utils import dict_utils

# from main_visualize_data import FrameControlButton, update_plot
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets 

In [3]:
import matplotlib 
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20)

In [23]:
scene_names = { 1: "TC_BGR_Intersection_VA",
                2: "DR_CHN_Merging_ZS",
                3: "DR_CHN_Roundabout_LN",
                4: "DR_DEU_Merging_MT",
                5: "DR_DEU_Roundabout_OF",
                6: "DR_USA_Intersection_EP0",
                7: "DR_USA_Intersection_EP1",
                8: "DR_USA_Intersection_GL",
                9: "DR_USA_Intersection_MA",
                10: "DR_USA_Roundabout_EP",
                11: "DR_USA_Roundabout_FT",
                12: "DR_USA_Roundabout_SR"}

## Selecting data file

In [24]:
scenario_name = scene_names[8]


track_file_number = 0
use_lanelet2_lib = False

In [25]:
error_string = ""
tracks_dir = "../recorded_trackfiles"
maps_dir = "../maps"
lanelet_map_ending = ".osm"
lanelet_map_file = maps_dir + "/" + scenario_name + lanelet_map_ending
scenario_dir = tracks_dir + "/" + scenario_name
track_file_prefix = "vehicle_tracks_"
track_file_ending = ".csv"
track_file_name = scenario_dir + "/" + track_file_prefix + str(track_file_number).zfill(3) + track_file_ending
if not os.path.isdir(tracks_dir):
    error_string += "Did not find track file directory \"" + tracks_dir + "\"\n"
if not os.path.isdir(maps_dir):
    error_string += "Did not find map file directory \"" + tracks_dir + "\"\n"
if not os.path.isdir(scenario_dir):
    error_string += "Did not find scenario directory \"" + scenario_dir + "\"\n"
if not os.path.isfile(lanelet_map_file):
    error_string += "Did not find lanelet map file \"" + lanelet_map_file + "\"\n"
if not os.path.isfile(track_file_name):
    error_string += "Did not find track file \"" + track_file_name + "\"\n"
if error_string != "":
    error_string += "Type --help for help."
    raise IOError(error_string)



In [26]:
track_dictionary = dataset_reader.read_tracks(track_file_name)

timestamp_min = 1e9
timestamp_max = 0
for key, track in dict_utils.get_item_iterator(track_dictionary):
    timestamp_min = min(timestamp_min, track.time_stamp_ms_first)
    timestamp_max = max(timestamp_max, track.time_stamp_ms_last)

In [27]:

# create a figure

def g(t, idxs): 
    fig = plt.figure(figsize=(15, 15))
    axes = plt.axes()

    # load and draw the lanelet2 map, either with or without the lanelet2 library
    lat_origin = 0.  # origin is necessary to correctly project the lat lon values in the osm file to the local
    lon_origin = 0.  # coordinates in which the tracks are provided; we decided to use (0|0) for every scenario
#     print("Loading map...")
    if use_lanelet2_lib:
        projector = lanelet2.projection.UtmProjector(lanelet2.io.Origin(lat_origin, lon_origin))
        laneletmap = lanelet2.io.load(lanelet_map_file, projector)
        map_vis_lanelet2.draw_lanelet_map(laneletmap, axes)
    else:
        map_vis_without_lanelet.draw_map_without_lanelet(lanelet_map_file, axes, lat_origin, lon_origin)


    # load the tracks
    track_dictionary = dataset_reader.read_tracks(track_file_name)

    timestamp_min = 1e9
    timestamp_max = 0
    for key, track in dict_utils.get_item_iterator(track_dictionary):
        timestamp_min = min(timestamp_min, track.time_stamp_ms_first)
        timestamp_max = max(timestamp_max, track.time_stamp_ms_last)
    start_timestamp = timestamp_min

    # storage for track visualization
    patches_dict = dict()
    text_dict = dict()


    # visualize tracks
    timestamp = t
    axes.set_title("\nts = {}".format(timestamp), fontsize=26)
    tracks_vis.update_objects_plot(timestamp, track_dictionary, patches_dict, text_dict, axes, idxs=idxs)
    plt.show()

interact(g, t=widgets.IntSlider(min=timestamp_min, max=timestamp_max, step=100, value=0), idxs=fixed([1,2]));


interactive(children=(IntSlider(value=100, description='t', max=300800, min=100, step=100), Output()), _dom_cl…